In [173]:
import tensorflow as tf 
from keras.layers import Dense,LSTM,Embedding,Bidirectional,Dropout
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer,one_hot
from keras.utils.np_utils import to_categorical   
import pandas as pd 
import numpy as np
from config import *
from keras import Sequential
from keras.initializers import Identity

import io

In [170]:
def sentence2vec(sentence,embedding_dictionary,stopwords,tokenizer):
    words = str(sentence).split()
    words = tokenizer(words)

    words = [w for w in words if w not in stopwords]

    M = []

    for w in words:
        if w in embedding_dictionary:
            M.append(embedding_dictionary[w])

    if len(M) == 0:
        return np.zeros(300)

    M = np.array(M)
    v = M.sum(axis = 0)

    return v / np.sqrt((v ** 2).sum())

def load_vectors(fname):
# taken from: https://fasttext.cc/docs/en/english-vectors.html
    fin = io.open(fname,'r',encoding='utf-8',newline='\n',errors='ignore')
   
    n, d = map(int, fin.readline().split())
    
    data = {}
    
    for line in fin:
    
        tokens = line.rstrip().split(' ')
        
        data[tokens[0]] = list(map(float, tokens[1:]))
    
    return data

In [175]:
embedding_matrix = load_vectors(r'C:\Users\Sidharth Tadeparti\Documents\DataScienceProjects\Project\input\wiki-news-300d-1M.vec')

FileNotFoundError: [Errno 2] No such file or directory: '.\\input\\wiki-news-300d-1M.vec'

In [ ]:
embedding_layer = Embedding(input_dim=1000001,
                                output_dim=300,
                                weights=[embedding_matrix],
                                input_length=sent_len,
                                trainable=False)

In [168]:
def lstm_model(voc_size, emb_size):
    model=Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(LSTM(100)))
    model.add(Dropout(0.3))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [156]:
def preprocess_data(data,fold):
    df_train = data[ data.kfold != fold]
    df_test = data[data.kfold==fold]

 
    tokenizer = Tokenizer(num_words=max_features, split=' ')
       
    tokenizer.fit_on_texts(data['review'].values)

    X_train = tokenizer.texts_to_sequences(df_train['review'].values)
    X_train = np.array(pad_sequences(X_train,maxlen=50))
    #X_train= to_categorical(X_train, num_classes=2)

    y_train = df_train['sentiment']

    X_test = tokenizer.texts_to_sequences(df_test['review'].values)
    X_test = np.array(pad_sequences(X_test,maxlen = 50))
    #X_test = to_categorical(X_test, num_classes=2)

    y_test = df_test['sentiment']
    

    return X_train,X_test,np.array(y_train),np.array(y_test)



In [104]:
DATA_FILE = (r"C:\Users\Sidharth Tadeparti\Documents\DataScienceProjects\Project\input\imdb_folds.csv")
data = pd.read_csv(DATA_FILE)


In [157]:
max_features = 5000
X_train,X_test,y_train,y_test = preprocess_data(data,0)

In [158]:
print(X_train[1])

[ 251   58  187    9 1212   11  189   57  341  166   15    9  978    8
   27    4  143 1036    4  369   11   13   20    1   30    1  167   55
   14  735   81   42    8  735    7    7  139  817 1896    2   63    3
  784 1197   17   42    1  144  735 1564]


In [169]:
model1 = lstm_model(X_train.shape[0],200)
sent_len = 50
print(model1.summary())

TypeError: __init__() got an unexpected keyword argument 'dtype'

In [ ]:
batch_size = 32
model1.fit(X_train, y_train, epochs = 7, batch_size=batch_size, verbose = 2,validation_data=[X_test,y_test])
